In [ ]:
pip install torch

In [ ]:
pip install torch_geometric

In [ ]:
pip install matplotlib

In [ ]:
pip install cvxpy

In [ ]:
import numpy as np
import torch
from torch_geometric.loader import  DataLoader
from torch_geometric.data import HeteroData, Data, Dataset, Batch
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
import torch.nn as nn
from torch.nn import Sequential as Seq, Linear as Lin, ReLU, ELU, Sigmoid, BatchNorm1d as BN, ReLU6 as ReLU6
import scipy.io
from torch_geometric.nn import HeteroConv, SAGEConv
import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils
import matplotlib.pyplot as plt
import cvxpy as cp

# Code generate data

In [ ]:
def generate_data(num, num_AP, num_UE, optimize = False):
    M = num_AP  # number of access points
    K = num_UE  # number of terminals
    D = 0.5  # in kilometer
    tau = 20  # training length
    U, S, V = np.linalg.svd(np.random.randn(tau, tau))
    B = 20  # MHz
    Hb = 15  # Base station height in m
    Hm = 1.65  # Mobile height in m
    f = 1900  # Frequency in MHz
    aL = (1.1 * np.log10(f) - 0.7) * Hm - (1.56 * np.log10(f) - 0.8)
    L = 46.3 + 33.9 * np.log10(f) - 13.82 * np.log10(Hb) - aL
    power_f = 0.1  # uplink power: 100 mW
    noise_p = 10 ** ((-203.975 + 10 * np.log10(20 * 10**6) + 9) / 10)  # noise power
    Pu = power_f / noise_p  # normalized receive SNR
    Pp = Pu  # pilot power
    sigma_shd = 8  # in dB
    D_cor = 0.1
    d0 = 0.01  # km
    d1 = 0.05  # km
    N = num  # realizations

    R_cf_min = np.zeros(N)
    R_cf_opt_min = np.zeros(N)
    R_cf_user = np.zeros((N, K))

    directs = np.zeros((N, K))  
    corsses = np.zeros((N, K, K)) 
    betas = np.zeros((N, M, K))  
    x_value = np.zeros((N, K))
    n = 0
    while n < N:
        valid = False  
        while not valid:  
            try:
                AP = np.zeros((M, 2, 9))
                AP[:, :, 0] = np.random.uniform(-D/2, D/2, (M, 2))

                D1 = np.zeros((M, 2))
                D1[:, 0] = D1[:, 0] + D*np.ones(M)
                AP[:, :, 1] = AP[:, :, 0] + D1

                D2 = np.zeros((M, 2))
                D2[:, 1] = D2[:, 1] + D*np.ones(M)
                AP[:, :, 2] = AP[:, :, 0] + D2

                D3 = np.zeros((M, 2))
                D3[:, 0] = D3[:, 0] - D*np.ones(M)
                AP[:, :, 3] = AP[:, :, 0] + D3

                D4 = np.zeros((M, 2))
                D4[:, 1] = D4[:, 1] - D*np.ones(M)
                AP[:, :, 4] = AP[:, :, 0] + D4

                D5 = np.zeros((M, 2))
                D5[:, 0] = D5[:, 0] + D*np.ones(M)
                D5[:, 1] = D5[:, 1] - D*np.ones(M)
                AP[:, :, 5] = AP[:, :, 0] + D5

                D6 = np.zeros((M, 2))
                D6[:, 0] = D6[:, 0] - D*np.ones(M)
                D6[:, 1] = D6[:, 1] - D*np.ones(M)
                AP[:, :, 6] = AP[:, :, 0] + D6

                D7 = np.zeros((M, 2))
                D7 = D7 + D*np.ones((M, 2))
                AP[:, :, 7] = AP[:, :, 0] + D7

                D8 = np.zeros((M, 2))
                D8 = D8 - D*np.ones((M, 2))
                AP[:, :, 8] = AP[:, :, 0] + D8

                # Initialize Ter positions
                Ter = np.zeros((K, 2, 9))
                Ter[:, :, 0] = np.random.uniform(-D/2, D/2, (K, 2))

                D1 = np.zeros((K, 2))
                D1[:, 0] = D1[:, 0] + D*np.ones(K)
                Ter[:, :, 1] = Ter[:, :, 0] + D1

                D2 = np.zeros((K, 2))
                D2[:, 1] = D2[:, 1] + D*np.ones(K)
                Ter[:, :, 2] = Ter[:, :, 0] + D2

                D3 = np.zeros((K, 2))
                D3[:, 0] = D3[:, 0] - D*np.ones(K)
                Ter[:, :, 3] = Ter[:, :, 0] + D3

                D4 = np.zeros((K, 2))
                D4[:, 1] = D4[:, 1] - D*np.ones(K)
                Ter[:, :, 4] = Ter[:, :, 0] + D4

                D5 = np.zeros((K, 2))
                D5[:, 0] = D5[:, 0] + D*np.ones(K)
                D5[:, 1] = D5[:, 1] - D*np.ones(K)
                Ter[:, :, 5] = Ter[:, :, 0] + D5

                D6 = np.zeros((K, 2))
                D6[:, 0] = D6[:, 0] - D*np.ones(K)
                D6[:, 1] = D6[:, 1] + D*np.ones(K)
                Ter[:, :, 6] = Ter[:, :, 0] + D6

                D7 = np.zeros((K, 2))
                D7 = D7 + D*np.ones((K, 2))
                Ter[:, :, 7] = Ter[:, :, 0] + D7

                D8 = np.zeros((K, 2))
                D8 = D8 - D*np.ones((K, 2))
                Ter[:, :, 8] = Ter[:, :, 0] + D8

                Dist = np.zeros((M, M))
                Cor = np.zeros((M, M))
                for m1 in range(M):
                    for m2 in range(M):
                        Dist[m1, m2] = np.min([np.linalg.norm(AP[m1, :, 0] - AP[m2, :, i]) for i in range(9)])
                        Cor[m1, m2] = np.exp(-np.log(2) * Dist[m1, m2] / D_cor)

                A1 = np.linalg.cholesky(Cor)
                x1 = np.random.randn(M, 1)
                sh_AP = A1 @ x1
                for m in range(M):
                    sh_AP[m] = (1/np.sqrt(2)) * sigma_shd * sh_AP[m] / np.linalg.norm(A1[m, :])

                Dist = np.zeros((K, K))
                Cor = np.zeros((K, K))
                for k1 in range(K):
                    for k2 in range(K):
                        Dist[k1, k2] = np.min([np.linalg.norm(Ter[k1, :, 0] - Ter[k2, :, i]) for i in range(9)])
                        Cor[k1, k2] = np.exp(-np.log(2) * Dist[k1, k2] / D_cor)

                A2 = np.linalg.cholesky(Cor)
                x2 = np.random.randn(K, 1)
                sh_Ter = A2 @ x2

                valid = True

            except np.linalg.LinAlgError:
                #print("Matrix not positive definite, retrying...")
                continue

        for k in range(K):
            sh_Ter[k] = (1/np.sqrt(2))*sigma_shd*sh_Ter[k]/np.linalg.norm(A2[k,:])

        Z_shd = np.zeros((M,K))
        for m in range(M):
            for k in range(K):
                Z_shd[m,k] = sh_AP[m] + sh_Ter[k]

        # Large-scale coefficients
        BETAA = np.zeros((M, K))
        dist = np.zeros((M,K))
        for m in range(M):
            for k in range(K):
                dist[m, k] = np.min([np.linalg.norm(AP[m,:,i]-Ter[k,:,0]) for i in range(9)])
                index = np.argmin([np.linalg.norm(AP[m,:,i]-Ter[k,:,0]) for i in range(9)])
                if dist[m, k] < d0:
                    betadB = -L - 35 * np.log10(d1) + 20 * np.log10(d1) - 20 * np.log10(d0)
                elif d0 <= dist[m, k] <= d1:
                    betadB = -L - 35 * np.log10(d1) + 20 * np.log10(d1) - 20 * np.log10(dist[m, k])
                else:
                    betadB = -L - 35 * np.log10(dist[m, k]) + Z_shd[m, k]
                BETAA[m, k] = 10 ** (betadB / 10)

        # Pilot assignment: (random choice)
        Phii = np.zeros((tau,K))
        for k in range(K):
            Point = k
            Phii[:,k] = U[:,Point]

        Phii_cf = Phii


        # Compute Gamma matrix
        Gammaa = np.zeros((M, K))
        mau = np.zeros((M,K))
        for m in range(M):
            for k in range(K):
                mau[m,k] = np.linalg.norm((BETAA[m,:]**(1/2)*(Phii_cf[:,k].T@Phii_cf)))**2
                Gammaa[m, k] = tau * Pp * BETAA[m, k]**2 / (tau * Pp * mau[m,k] + 1)

        # SINR and rate calculation
        SINR = np.zeros(K)
        R_cf = np.zeros(K)

        PC = np.zeros((K,K))
        for ii in range(K):
            for k in range(K):
                PC[ii,k] = np.sum((Gammaa[:,k]/BETAA[:,k]*BETAA[:,ii])*(Phii_cf[:,k].T@Phii_cf[:,ii]))
        PC1 = (np.abs(PC))**2
        
        for k in range(K):
            deno1 = 0
            for m in range(M):
                deno1 += Gammaa[m,k]*np.sum(BETAA[m,:])
            
            SINR[k] = Pu*(np.sum(Gammaa[:,k]))**2/(np.sum(Gammaa[:,k]) + Pu*deno1 + Pu*np.sum(PC1[:,k]) - Pu*PC1[k,k])
            R_cf[k] = np.log2(1+SINR[k])

        stepp = 5
        Ratestep = np.zeros((stepp, K))
        Ratestep[0,:] = R_cf

        for st in range(1, stepp): 
            minvalue, minindex = np.min(Ratestep[st - 1, :]), np.argmin(Ratestep[st - 1, :])
            Mat = np.zeros((tau, tau)) - Pu * np.sum(BETAA[:, minindex]) * np.outer(Phii_cf[:, minindex], Phii_cf[:, minindex])
            for kk in range(K):
                Mat += Pu * np.sum(BETAA[:, kk]) * (Phii_cf[:, kk]@Phii_cf[:, kk].T)
        
            U1, S1, V1 = np.linalg.svd(Mat, full_matrices=True)
            Phii_cf[:, minindex] = U1[:, tau-1]


        Gammaa = np.zeros((M, K))
        mau = np.zeros((M, K))

        for m in range(M):
            for k in range(K):
                mau[m, k] = np.linalg.norm(
                    (BETAA[m, :]**0.5) * (Phii_cf[:, k].T@Phii_cf)**2
                )**2
        for m in range(M):
            for k in range(K):
                Gammaa[m, k] = tau * Pp * BETAA[m, k]**2 / (tau * Pp * mau[m, k] + 1)

        SINR = np.zeros(K)
        PC = np.zeros((K, K))

        for ii in range(K):
            for k in range(K):
                PC[ii, k] = np.sum((Gammaa[:, k] / BETAA[:, k]) * BETAA[:, ii]) * np.dot(Phii_cf[:, k], Phii_cf[:, ii])
        PC1 = np.abs(PC)**2
        for k in range(K):
            deno1 = 0
            for m in range(M):
                deno1 += Gammaa[m, k] * np.sum(BETAA[m, :])
            SINR[k] = Pu * (np.sum(Gammaa[:, k]))**2 / (
                np.sum(Gammaa[:, k]) + Pu * deno1 + Pu * np.sum(PC1[:, k]) - Pu * PC1[k, k]
            )
            # Rate calculation
            Ratestep[st-1, k] = np.log2(1 + SINR[k])

        R_cf_min[n] = np.min(Ratestep[stepp-1, :])
        R_cf_user[n,:] = Ratestep[stepp-1, :]

        tmin = 2**R_cf_min[n]-1
        tmax=2**(2*R_cf_min[n]+1.2)-1
        epsi = max(tmin/5,0.01)

        BETAAn = BETAA * Pu
        Gammaan = Gammaa * Pu
        PhiPhi = np.zeros((K, K))
        Te1 = np.zeros((K, K))
        Te2 = np.zeros((K, K))
        direct = np.zeros(K)

        for ii in range(K):
            for k in range(K):
                PhiPhi[ii, k] = np.linalg.norm(Phii_cf[:, ii].T@Phii_cf[:, k])

        for ii in range(K):
            direct[ii] = np.sum(Gammaan[:, ii])
            for k in range(K):
                Te1[ii, k] = np.sum(BETAAn[:, ii] * Gammaan[:, k])
                Te2[ii, k] = (
                    np.sum((Gammaan[:, k] / BETAA[:, k]) * BETAA[:, ii])**2
                    * PhiPhi[k, ii]**2
                )
                if ii == k:
                    Te2[ii, k] = 0


        # Max-min power allocation using CVXPY
        if optimize:
            x = cp.Variable(K)
            tmin = 2**R_cf_min[n] - 1
            tmax = 2**(2 * R_cf_min[n] + 1.2) - 1
            epsi = max(tmin / 5, 0.05)

            while tmax - tmin > epsi:
                tnext = (tmax + tmin) / 2

                x = cp.Variable(K)
                constraints = []
                for k in range(K):
                    left_term = Te1[:, k].T @ x

                    if k > 0:
                        top_part = Te2[:k, k]
                        top_x = x[:k]
                    else:
                        top_part = np.array([])  
                        top_x = np.array([])

                    if k < K - 1:
                        bottom_part = Te2[k+1:, k]
                        bottom_x = x[k+1:]
                    else:
                        bottom_part = np.array([])
                        bottom_x = np.array([])

                    if top_part.size > 0 and bottom_part.size > 0:
                        middle_term = cp.hstack([top_part, bottom_part]).T @ cp.hstack([top_x, bottom_x])
                    elif top_part.size > 0:
                        middle_term = top_part.T @ top_x
                    elif bottom_part.size > 0:
                        middle_term = bottom_part.T @ bottom_x
                    else:
                        middle_term = 0
                    
                    right_term = (1 / tnext) * (np.sum(Gammaan[:, k]))**2 * x[k]
                    constraints.append(left_term + middle_term + np.sum(Gammaan[:, k]) <= right_term)

                constraints += [x >= 0, x <= 1]
        
                prob = cp.Problem(cp.Minimize(0), constraints)
                prob.solve()

                if prob.status == cp.OPTIMAL:
                    #print(f"Problem is feasible at tnext = {tnext}")
                    tmin = tnext
                else:
                    #print(f"Problem not feasible at tnext = {tnext}")
                    tmax = tnext
            if(x.value is None):
                continue
            x_value[n] = x.value
            R_cf_opt_min[n] = np.log2(1 + tmin)
        cross = Te1 + Te2
        directs[n, :] = direct
        corsses[n, :, :] = cross
        betas[n, :, :] = BETAAn
        n += 1
    if optimize:
        return betas, directs, corsses, R_cf_opt_min, x_value
    return betas, directs, corsses

In [ ]:
train_layouts = 10000
test_layouts = 200
M = 30
K = 6

# Generate train data set

In [ ]:
beta_train , direct_train, cross_train = generate_data(train_layouts, M, K, optimize=False)

# Generate test data set

In [ ]:
beta_test, direct_test, corsses_test, R_cf_opt_min, x_value = generate_data(test_layouts, M, K, optimize=True)

In [ ]:
cross_train = cross_train.transpose(0,2,1)
cross_test = corsses_test.transpose(0,2,1)

In [ ]:
def normalize_data(train_data,test_data):
    train_mean = np.mean(train_data)
    train_std = np.std(train_data)
    norm_train = (train_data)/train_std
    norm_test = (test_data)/train_std
    n1, n2 = norm_train.shape[0], norm_test.shape[0]
    return norm_train, norm_test
norm_train_losses, norm_test_losses = normalize_data(beta_train**(1/2), beta_test**(1/2) )

In [ ]:
class PCDataset(Dataset):
    def __init__(self, norm_losses, direct, cross, KM):
        self.norm_losses = norm_losses
        self.direct = torch.tensor(direct, dtype=torch.float)
        self.cross = torch.tensor(cross, dtype=torch.float)
        self.KM = KM
        self.get_cg()
        self.process()

    def build_graph(self, idx):
        edge_feature = self.norm_losses[idx, :, :].reshape((self.KM[0] * self.KM[1], 1), order='F')
        edge_feature = np.concatenate((edge_feature, np.ones_like(edge_feature)), axis=-1)
        edge_feature = torch.tensor(edge_feature, dtype=torch.float)

        edge_index = torch.tensor(self.adj, dtype=torch.long).t().contiguous()
        edge_index_t = torch.tensor(self.adj_t, dtype=torch.long).t().contiguous()
        ue_features = torch.ones((self.KM[0], 1))
        ap_features = torch.ones((self.KM[1], 1))

        data = HeteroData()
        data['UE'].x = ue_features
        data['AP'].x = ap_features
        data['UE', 'com-by', 'AP'].edge_index = edge_index
        data['UE', 'com-by', 'AP'].edge_attr = edge_feature
        data['AP', 'com', 'UE'].edge_index = edge_index_t
        data['AP', 'com', 'UE'].edge_attr = edge_feature

        return data

    def get_cg(self):
        self.adj = []
        self.adj_t = []
        for i in range(self.KM[0]):
            for j in range(self.KM[1]):
                self.adj.append([i, j])
                self.adj_t.append([j, i])

    def process(self):
        self.graph_list = [self.build_graph(i) for i in range(len(self.direct))]

    def __len__(self):
        return len(self.direct)

    def __getitem__(self, index):
        return self.graph_list[index], self.direct[index], self.cross[index]

In [ ]:
def collate(samples):
    '''Pytorch Geometric collate function'''
    graphs, direct, cross = map(list, zip(*samples))
    batched_graph = Batch.from_data_list(graphs)
    return batched_graph, torch.stack(direct), torch.stack(cross)

In [ ]:
train_data = PCDataset(norm_train_losses, direct_train, cross_train, (K, M))
test_data = PCDataset(norm_test_losses, direct_test, cross_test, (K, M))

# Mục mới

In [ ]:
batch_size = 64
train_loader = DataLoader(train_data, batch_size, shuffle=True, collate_fn=collate)
test_loader = DataLoader(test_data, test_layouts, shuffle=False, collate_fn=collate)

In [ ]:
def rate_loss(allocs, directlink_channel_losses, crosslink_channel_losses, test_mode = False):
    SINRs_numerators = allocs * directlink_channel_losses**2
    SINRs_denominators = torch.squeeze(torch.matmul(crosslink_channel_losses, torch.unsqueeze(allocs, axis=-1))) + directlink_channel_losses
    SINRs = SINRs_numerators / SINRs_denominators
    rates = torch.log2(1 + SINRs)
    min_rate = torch.min(rates, dim = 1)[0] # take min
    if test_mode:
        return min_rate
    else:
        return -torch.mean(min_rate)

In [ ]:
def MLP(channels, batch_norm=True):
    return Seq(*[
        Seq(Lin(channels[i - 1], channels[i]), ReLU(), BN(channels[i]))
        for i in range(1, len(channels))
    ])
class EdgeConv(MessagePassing):
    def __init__(self, input_dim, node_dim, **kwargs):
        super(EdgeConv, self).__init__(aggr='mean')  # mean aggregation
        self.lin = MLP([input_dim, 32])
        self.res_lin = Lin(node_dim, 32)
        self.bn = BN(32)

    def forward(self, x, edge_index, edge_attr):

        feat_src, feat_dst = x


        out = self.propagate(edge_index=edge_index, x=(feat_src, feat_dst), edge_attr=edge_attr)


        return self.bn(out + self.res_lin(feat_dst))

    def message(self, x_j, x_i, edge_attr):
        # Tạo ra thông điệp từ các nút nguồn, nút đích và đặc tính cạnh
        out = torch.cat([x_j, x_i, edge_attr], dim=1)
        return self.lin(out)

    def update(self, aggr_out):
        # Cập nhật giá trị nút đích sau khi tập hợp
        return aggr_out



In [ ]:
class RGCN(nn.Module):
    def __init__(self):
        super(RGCN, self).__init__()
        self.conv1 = HeteroConv({
            ('UE', 'com-by', 'AP'): EdgeConv(4, 1),
            ('AP', 'com', 'UE'): EdgeConv(4, 1)
        }, aggr='mean')

        self.conv2 = HeteroConv({
            ('UE', 'com-by', 'AP'): EdgeConv(66, 32),
            ('AP', 'com', 'UE'): EdgeConv(66, 32)
        }, aggr='mean')

        self.conv3 = HeteroConv({
            ('UE', 'com-by', 'AP'): EdgeConv(66, 32),
            ('AP', 'com', 'UE'): EdgeConv(66, 32)
        }, aggr='mean')

        self.mlp = MLP([32, 16])
        self.mlp = nn.Sequential(*[self.mlp, Seq(Lin(16, 1), Sigmoid())])

    def forward(self,x_dict, edge_index_dict, edge_attr_dict):
        out = self.conv1(x_dict, edge_index_dict, edge_attr_dict)
        out = self.conv2(out, edge_index_dict, edge_attr_dict)
        out = self.conv3(out, edge_index_dict, edge_attr_dict)
        out = self.mlp(out['UE'])
        return out


In [ ]:
model = RGCN()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.8)

In [ ]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {trainable_params}")

In [ ]:
def train_model(epoch, model, optimizer, train_loader):
    """ Train for one epoch. """
    model.train()
    loss_all = 0
    for batch_idx, (data, d_train, c_train) in enumerate(train_loader):
        K = d_train.shape[-1]
        n = len(data['UE'].x)
        bs = len(data['UE'].x) // K

        optimizer.zero_grad()

        # Lấy các đặc trưng nút từ từ điển x_dict
        user_feats = data['AP'].x
        item_feats = data['UE'].x
        node_features = {'AP': user_feats, 'UE': item_feats}

        # Truyền qua mô hình, bao gồm cả edge_attr_dict
        output = model(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs, -1)

        # Tính loss và thực hiện backpropagation
        loss = rate_loss(output, d_train, c_train)
        loss.backward()

        loss_all += loss.item() * bs
        optimizer.step()

    return loss_all / len(train_loader.dataset)

def test_model(loader, model):
    model.eval()
    correct = 0
    with torch.no_grad():
        for (data, d_test, c_test) in loader:
            K = d_test.shape[-1]
            n = len(data['UE'].x)
            bs = len(data['UE'].x) // K

            # Lấy các đặc trưng nút từ từ điển x_dict
            user_feats = data['AP'].x
            item_feats = data['UE'].x
            # Create a dictionary for node features
            node_features = {'AP': user_feats, 'UE': item_feats}

            output = model(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs, -1)

            # Tính loss
            loss = rate_loss(output, d_test, c_test)
            correct += loss.item() * bs

    return correct / len(loader.dataset)

In [ ]:
record = []

for epoch in range(0, 30):
    if epoch % 1 == 0:
        with torch.no_grad():
            test_rate = test_model(test_loader, model)
            train_rate = test_model(train_loader, model)
        print(f'Epoch {epoch:03d}, Train Rate: {train_rate:.4f}, Test Rate: {test_rate:.4f}')
        record.append([train_rate, test_rate])

    train_model(epoch, model, optimizer, train_loader )
    scheduler.step()


In [ ]:
import time
gnn_rates = None
start_time = time.time()

for (data, d_test, c_test) in test_loader:
    K = d_test.shape[-1]
    n = len(data['UE'].x)
    bs = len(data['UE'].x) // K
    user_feats = data['AP'].x
    item_feats = data['UE'].x
    node_features = {'AP': user_feats, 'UE': item_feats}
    output = model(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs,-1)
    gnn_rates = rate_loss(output, d_test, c_test, True).flatten().detach().numpy()
    
end_time = time.time()

print("Time taken for gnn model:", (end_time - start_time)/test_layouts, "seconds")

In [ ]:
np.mean(gnn_rates)

In [ ]:
test_model(test_loader, model)

# Quantum MLP

# HQGNN Amplitude Embedding

In [ ]:
pip install pennylane

In [ ]:
import pennylane as qml
from pennylane import numpy as np

n_qubits = 5
dev = qml.device('default.qubit', wires=n_qubits)


In [ ]:
n_layers_circuit_X = 2
def circuit_X_entangling(weights, n_qubits):
    qml.CRX(weights[-1], wires=[n_qubits-1, 0])
    for i in range(n_qubits-1):
        qml.CRX(weights[i], wires=[i, (i+1)])

@qml.qnode(dev, interface='torch')
def circuit_X(inputs, layer_weights):
    qml.AmplitudeEmbedding(features=inputs, wires=range(n_qubits), normalize=True, pad_with=0.)
    for l in range(n_layers_circuit_X):
        circuit_X_entangling(layer_weights[l], n_qubits)
    return qml.probs(wires=range(n_qubits))
weight_shapes_circuit_X = { "layer_weights": (n_layers_circuit_X, n_qubits)}

In [ ]:
n_layers_circuit_Z = 2 
def circuit_Z_entangling(weights, n_qubits):
    qml.CRZ(weights[-1], wires=[n_qubits - 1, 0])
    for i in range(n_qubits - 1):
        qml.CRZ(weights[i], wires=[i, i + 1])

@qml.qnode(dev, interface='torch')
def circuit_Z(inputs, layer_weights):
    qml.AmplitudeEmbedding(features=inputs, wires=range(n_qubits), normalize=True, pad_with=0.)
    for l in range(n_layers_circuit_Z):
        circuit_Z_entangling(layer_weights[l], n_qubits)
    return qml.probs(wires=range(n_qubits))
weight_shapes_circuit_Z = { "layer_weights": (n_layers_circuit_Z, n_qubits)}

In [ ]:
def MLP(channels, batch_norm=True):
    return Seq(*[
        Seq(Lin(channels[i - 1], channels[i]), ReLU(), BN(channels[i]))
        for i in range(1, len(channels))
    ])
class Q_layer(MessagePassing):
    def __init__(self,src_dim, dst_dim, edge_dim, **kwargs):
        super(Q_layer, self).__init__(aggr='mean')  # mean aggregation
        self.lin_res = qml.qnn.TorchLayer(circuit_Z, weight_shapes_circuit_Z)
        self.lin_qml = qml.qnn.TorchLayer(circuit_X, weight_shapes_circuit_X)
        self.in_linear = nn.Linear(src_dim + dst_dim + edge_dim, 2 ** n_qubits)
        self.bn = BN(2 ** n_qubits)

    def forward(self, x, edge_index, edge_attr):
        feat_src, feat_dst = x
        out = self.propagate(edge_index=edge_index, x=(feat_src, feat_dst), edge_attr=edge_attr)
        out = out + self.lin_res(feat_dst)
        return self.bn(out)

    def message(self, x_j, x_i, edge_attr):
        out = torch.cat([x_j, x_i, edge_attr], dim=1)
        out = self.in_linear(out)
        out = self.lin_qml(out)
        return out

    def update(self, aggr_out):
        return aggr_out

In [ ]:
class RGCN_Hybrid_mid(nn.Module):
    def __init__(self):
        super(RGCN_Hybrid_mid, self).__init__()
        out_dim = 2**n_qubits
        edge_dim = 2

        self.conv1 = HeteroConv({
            ('UE', 'com-by', 'AP'): Q_layer(1, 1, edge_dim),
            ('AP', 'com', 'UE'): Q_layer(1, 1, edge_dim,)
        }, aggr='mean')

        self.conv2 = HeteroConv({
            ('UE', 'com-by', 'AP'): Q_layer(out_dim, out_dim, edge_dim),
            ('AP', 'com', 'UE'): Q_layer(out_dim, out_dim, edge_dim)
        }, aggr='mean')

        self.conv3 = HeteroConv({
             ('UE', 'com-by', 'AP'): Q_layer(out_dim, out_dim, edge_dim),
             ('AP', 'com', 'UE'): Q_layer(out_dim, out_dim, edge_dim)
         }, aggr='mean')


        
        self.mlp = MLP([32, 16])
        self.mlp = nn.Sequential(*[self.mlp, Seq(Lin(16, 1), Sigmoid())])
    def forward(self, x_dict, edge_index_dict, edge_attr_dict):
        out = self.conv1(x_dict, edge_index_dict, edge_attr_dict)
        out = self.conv2(out, edge_index_dict, edge_attr_dict)
        out = self.conv3(out, edge_index_dict, edge_attr_dict)
        out = self.mlp(out['UE'])
        return out

In [ ]:
model_qml_amplitude = RGCN_Hybrid_mid().to()

optimizer_qml_amplitude = torch.optim.Adam(model_qml_amplitude.parameters(), lr=5e-4)
scheduler_qml_amplitude = torch.optim.lr_scheduler.StepLR(optimizer_qml_amplitude, step_size=10, gamma=0.65)

In [ ]:
trainable_params = sum(p.numel() for p in model_qml_amplitude.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {trainable_params}")

In [ ]:
record_edge = []

for epoch in range(0, 30):

    with torch.no_grad():
        test_rate = test_model(test_loader, model_qml_amplitude)
        train_rate = test_model(train_loader, model_qml_amplitude)
        record_edge.append([train_rate, test_rate])
    if epoch % 1 == 0:
        print(f'Epoch {epoch:02d}, Train Rate: {train_rate:.4f}, Test Rate: {test_rate:.4f}')
    train_model(epoch, model_qml_amplitude, optimizer_qml_amplitude, train_loader)
    scheduler_qml_amplitude.step()

In [ ]:
gnn_q_rates = None
gnn_rates = None
for (data, d_test, c_test) in test_loader:
    K = d_test.shape[-1]
    n = len(data['UE'].x)
    bs = len(data['UE'].x) // K
    user_feats = data['AP'].x
    item_feats = data['UE'].x
    node_features = {'AP': user_feats, 'UE': item_feats}
    output = model_qml_amplitude(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs,-1)
    output_2 = model(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs,-1)
    full = torch.ones_like(output)
    all_one_rates = rate_loss(full, d_test, c_test, True).flatten().detach().numpy()
    gnn_q_rates = rate_loss(output, d_test, c_test, True).flatten().detach().numpy()
    gnn_rates = rate_loss(output_2, d_test, c_test, True).flatten().detach().numpy()
test_data = scipy.io.loadmat('cf_test_6_30.mat')
opt_rates = test_data['R_cf_opt_min']

In [ ]:
min_rate, max_rate = 0, 2
y_axis = np.arange(0, 1.0, 1/202)
opt_rates.sort();gnn_rates.sort(); all_one_rates.sort();gnn_q_rates.sort()
gnn_q_rates = np.insert(gnn_q_rates, 0, min_rate); gnn_q_rates = np.insert(gnn_q_rates,201,max_rate)
gnn_rates = np.insert(gnn_rates, 0, min_rate); gnn_rates = np.insert(gnn_rates,201,max_rate)
all_one_rates = np.insert(all_one_rates, 0, min_rate); all_one_rates = np.insert(all_one_rates,201,max_rate)
opt_rates = np.insert(opt_rates, 0, min_rate); opt_rates = np.insert(opt_rates,201,max_rate)

In [ ]:
plt.plot(gnn_rates, y_axis, label = 'GNN')
#plt.plot(gnn_rates_qml_mid_angle, y_axis, label = 'HQGNN angle')
plt.plot(gnn_q_rates, y_axis, label = 'HQGNN amplitude')
plt.plot(opt_rates, y_axis, label = 'Optimal')
plt.plot(all_one_rates, y_axis, label = 'Maximum Power')
plt.xlabel('Minimum rate [bps/Hz]', {'fontsize':16})
plt.ylabel('Empirical CDF', {'fontsize':16})
plt.legend(fontsize = 12)
plt.grid()

In [ ]:
print(f'QGNN in middle: {np.mean(gnn_q_rates)}')
print(f'GNN: {np.mean(gnn_rates)}')

In [ ]:
gnn_test_rates = None
gnn_test_rates_q = None
for (data, d_test, c_test) in test_loader:
    K = d_test.shape[-1]
    n = len(data['UE'].x)
    bs = len(data['UE'].x) // K
    user_feats = data['AP'].x
    item_feats = data['UE'].x
    node_features = {'AP': user_feats, 'UE': item_feats}
    output_qgnn = model_qml_amplitude(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs,-1)
    output_gnn = model(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs, -1)
    gnn_test_rates = rate_loss(output_gnn, d_test, c_test, True).flatten().detach().numpy()
    gnn_test_rates_q = rate_loss(output_qgnn, d_test, c_test, True).flatten().detach().numpy()

In [ ]:
count = np.sum(gnn_test_rates_q > gnn_test_rates)
print(f'Number of samples where HQGNN outperforms GNN: {count}')

In [ ]:
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams.update({'font.size': 12})
fig, ax = plt.subplots(figsize=(6, 6), dpi=180)
epsilon = 0.001 

positions_greater = np.where((gnn_test_rates_q - gnn_test_rates) > epsilon)[0]
positions_equal = np.where(np.abs(gnn_test_rates_q - gnn_test_rates) <= epsilon)[0]

count_greater = len(positions_greater)
count_equal = len(positions_equal)
count_not_greater = 200 - count_greater - count_equal

labels = ['HQGNN amplitude > GNN', 'HQGNN amplitude ≈ GNN', 'HQGNN angle < GNN']
sizes = [count_greater, count_equal, count_not_greater]
explode = (0.1, 0.1, 0)  

# Remove labels in the pie chart but keep legend
ax.pie(sizes, explode=explode, labels=None, autopct='%1.1f%%', shadow=True, startangle=140)

# Add legend
ax.legend(labels, loc='upper left')
ax.grid()


In [ ]:
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams.update({'font.size': 12})
fig, ax = plt.subplots(figsize=(6, 6), dpi=180)
epsilon = 0.001 

positions_greater = np.where((gnn_test_rates_q - gnn_test_rates) > epsilon)[0]
positions_equal = np.where(np.abs(gnn_test_rates_q - gnn_test_rates) <= epsilon)[0]


count_greater = len(positions_greater)
count_equal = len(positions_equal)
count_not_greater = 200 - count_greater - count_equal


labels = ['HQGNN amplitude > GNN', 'HQGNN amplitude ≈ GNN', 'HQGNN amplitude < GNN']
sizes = [count_greater, count_equal, count_not_greater]
explode = (0.1, 0.1, 0)  

ax.pie(sizes, explode=explode, labels=None, autopct='%1.1f%%', shadow=True, startangle=140)
plt.title('Ratio of HQGNN amplitude position to GNN with error 1e-3')
ax.legend(labels, loc='upper left')
plt.show()


In [ ]:
data = scipy.io.loadmat('cf_train_5_20_test.mat')
len(data['betas'])

In [ ]:
gnn_test_rates = None
gnn_test_rates_q = None
positions_greater = 0
positions_equal = 0
positions_less = 0
epsilon = 0.001 
num = 1000
file_name = ['cf_train_5_20_test.mat', 'cf_train_10_30_test.mat', 'cf_train_20_40_test.mat', 'cf_train_20_50_test.mat']
#file_name = ['cf_test_6_30.mat']
scenarior = [(5, 20), (10, 30), (20, 40), (20, 50)]

for i in range(len(file_name)):
    data = scipy.io.loadmat(file_name[i])
    beta = data['betas'][:num]
    direct = data['directs'][:num]
    cross = data['corsses'][:num].transpose(0, 2, 1)
    
    _, norm_losses = normalize_data(beta**(1/2), beta**(1/2))
    data = PCDataset(norm_losses, direct, cross, scenarior[i])
    loader = DataLoader(data, 100, shuffle=False, collate_fn=collate)
    
    for batch_idx, (g, d_test, c_test) in enumerate(loader):
        K = d_test.shape[-1]
        n = len(g['UE'].x)
        bs = len(g['UE'].x) // K
        user_feats = g['AP'].x
        item_feats = g['UE'].x
        node_features = {'AP': user_feats, 'UE': item_feats}
   
        output_qgnn_mid = model_qml_amplitude(node_features, g.edge_index_dict, g.edge_attr_dict).reshape(bs, -1)
        output_gnn = model(node_features, g.edge_index_dict, g.edge_attr_dict).reshape(bs, -1)

        gnn_test_rates = rate_loss(output_gnn, d_test, c_test, True).flatten().detach().numpy()
        gnn_test_rates_q = rate_loss(output_qgnn_mid, d_test, c_test, True).flatten().detach().numpy()
        count_greater = np.sum((gnn_test_rates_q - gnn_test_rates) > epsilon)
        count_equal = np.sum(np.abs(gnn_test_rates_q - gnn_test_rates) <= epsilon)
        positions_greater += count_greater
        positions_equal += count_equal
        batch_size = len(gnn_test_rates)
        positions_less += 100 - count_greater - count_equal

In [ ]:
print(positions_greater,positions_equal,  positions_less)

In [ ]:
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams.update({'font.size': 12})
fig, ax = plt.subplots(figsize=(6, 6), dpi=180)
labels = ['HQGNN amplitude > GNN', 'HQGNN amplitude ≈ GNN', 'HQGNN amplitude < GNN']
sizes = [positions_greater, positions_equal, positions_less]
explode = (0.1, 0.1, 0)  


ax.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%', shadow=True, startangle=140)
ax.legend()
plt.title(f'Error {epsilon}')
plt.show()


# Angle Embedding

In [ ]:
import pennylane as qml
from pennylane import numpy as np

n_qubits = 5
dev = qml.device('default.qubit', wires=n_qubits)


In [ ]:
n_layers_circuit_X2 = 1
def circuit_X2_entangling(weights, n_qubits):
    qml.CRX(weights[-1], wires=[n_qubits-1, 0])
    for i in range(n_qubits-1):
        qml.CRX(weights[i], wires=[i, (i+1)])

@qml.qnode(dev, interface='torch')
def circuit_X2(inputs, layer_weights):
    qml.AngleEmbedding(features=inputs, wires=range(n_qubits))
    for l in range(n_layers_circuit_X2):
        circuit_X2_entangling(layer_weights[l], n_qubits)
    return qml.probs(wires=range(n_qubits))
weight_shapes_circuit_X2 = { "layer_weights": (n_layers_circuit_X2, n_qubits)}

In [ ]:
n_layers_circuit_Z2 = 1
def circuit_Z2_entangling(weights, n_qubits):
    qml.CRZ(weights[-1], wires=[n_qubits - 1, 0])
    for i in range(n_qubits - 1):
        qml.CRZ(weights[i], wires=[i, i + 1])

@qml.qnode(dev, interface='torch')
def circuit_Z2(inputs, layer_weights):
    qml.AngleEmbedding(features=inputs, wires=range(n_qubits))
    for l in range(n_layers_circuit_Z2):
        circuit_Z2_entangling(layer_weights[l], n_qubits)
    return qml.probs(wires=range(n_qubits))
weight_shapes_circuit_Z2 = { "layer_weights": (n_layers_circuit_Z2, n_qubits)}

In [ ]:
def MLP(channels, batch_norm=True):
    return Seq(*[
        Seq(Lin(channels[i - 1], channels[i]), ReLU(), BN(channels[i]))
        for i in range(1, len(channels))
    ])
class Q_layer_angle(MessagePassing):
    def __init__(self,src_dim, dst_dim, edge_dim, out_dim, **kwargs):
        super(Q_layer_angle, self).__init__(aggr='mean')  
        self.lin_res = qml.qnn.TorchLayer(circuit_X2, weight_shapes_circuit_X2)
        self.lin_qml = qml.qnn.TorchLayer(circuit_Z2, weight_shapes_circuit_Z2)
        self.in_linear = nn.Linear(src_dim + dst_dim + edge_dim, out_dim)
        self.linear = nn.Linear(dst_dim, n_qubits)
        self.bn = BN(32)

    def forward(self, x, edge_index, edge_attr):
        feat_src, feat_dst = x
        out = self.propagate(edge_index=edge_index, x=(feat_src, feat_dst), edge_attr=edge_attr)
        out = out + self.lin_res(self.linear(feat_dst))
        return self.bn(out)

    def message(self, x_j, x_i, edge_attr):
        out = torch.cat([x_j, x_i, edge_attr], dim=1)
        out = self.in_linear(out)
        out = self.lin_qml(out)
        return out

    def update(self, aggr_out):
        return aggr_out

In [ ]:
class RGCN_Hybrid_mid_angle(nn.Module):
    def __init__(self):
        super(RGCN_Hybrid_mid_angle, self).__init__()
        out_dim = n_qubits
        edge_dim = 2

        self.conv1 = HeteroConv({
            ('UE', 'com-by', 'AP'): Q_layer_angle(1, 1, edge_dim, out_dim),
            ('AP', 'com', 'UE'): Q_layer_angle(1, 1, edge_dim,out_dim)
        }, aggr='mean')

        self.conv2 = HeteroConv({
            ('UE', 'com-by', 'AP'): Q_layer_angle(32, 32, edge_dim, out_dim),
            ('AP', 'com', 'UE'): Q_layer_angle(32, 32, edge_dim, out_dim)
        }, aggr='mean')

        self.conv3 = HeteroConv({
             ('UE', 'com-by', 'AP'): Q_layer_angle(32, 32, edge_dim, out_dim),
             ('AP', 'com', 'UE'): Q_layer_angle(32, 32, edge_dim, out_dim)
         }, aggr='mean')

        self.mlp = MLP([32, 16])
        self.mlp = nn.Sequential(*[self.mlp, Seq(Lin(16, 1), Sigmoid())])
    def forward(self, x_dict, edge_index_dict, edge_attr_dict):
        out = self.conv1(x_dict, edge_index_dict, edge_attr_dict)
        out = self.conv2(out, edge_index_dict, edge_attr_dict)
        out = self.conv3(out, edge_index_dict, edge_attr_dict)
        out = self.mlp(out['UE'])
        return out

In [ ]:
model_qml_mid_angle = RGCN_Hybrid_mid_angle().to()

optimizer_qml_mid_angle = torch.optim.Adam(model_qml_mid_angle.parameters(), lr=5e-4)
scheduler_qml_mid_angle = torch.optim.lr_scheduler.StepLR(optimizer_qml_mid_angle, step_size=10, gamma=0.7)

In [ ]:
trainable_params = sum(p.numel() for p in model_qml_mid_angle.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {trainable_params}")

In [ ]:
record_edge = []

for epoch in range(0, 30):

    with torch.no_grad():
        test_rate = test_model(test_loader, model_qml_mid_angle)
        train_rate = test_model(train_loader, model_qml_mid_angle)
        record_edge.append([train_rate, test_rate])
    if epoch % 1 == 0:
        print(f'Epoch {epoch:02d}, Train Rate: {train_rate:.4f}, Test Rate: {test_rate:.4f}')
    train_model(epoch, model_qml_mid_angle, optimizer_qml_mid_angle, train_loader)
    scheduler_qml_mid_angle.step()

In [ ]:
for (data, d_test, c_test) in test_loader:
    K = d_test.shape[-1]
    n = len(data['UE'].x)
    bs = len(data['UE'].x) // K
    user_feats = data['AP'].x
    item_feats = data['UE'].x
    node_features = {'AP': user_feats, 'UE': item_feats}
    output_qgnn = model_qml_mid_angle(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs,-1)
    output_gnn = model(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs, -1)
    gnn_test_rates = rate_loss(output_gnn, d_test, c_test, True).flatten().detach().numpy()
    gnn_test_rates_q = rate_loss(output_qgnn, d_test, c_test, True).flatten().detach().numpy()

In [ ]:
np.mean(gnn_test_rates_q)

In [ ]:
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams.update({'font.size': 12})
fig, ax = plt.subplots(figsize=(6, 6), dpi=180)
epsilon = 0.001 

positions_greater = np.where((gnn_test_rates_q - gnn_test_rates) > epsilon)[0]
positions_equal = np.where(np.abs(gnn_test_rates_q - gnn_test_rates) <= epsilon)[0]

count_greater = len(positions_greater)
count_equal = len(positions_equal)
count_not_greater = 200 - count_greater - count_equal

labels = ['HQGNN angle > GNN', 'HQGNN angle ≈ GNN', 'HQGNN angle < GNN']
sizes = [count_greater, count_equal, count_not_greater]
explode = (0.1, 0.1, 0)  

# Remove labels in the pie chart but keep legend
ax.pie(sizes, explode=explode, labels=None, autopct='%1.1f%%', shadow=True, startangle=140)

# Add legend
ax.legend(labels, loc='upper left')
ax.grid()


In [ ]:
gnn_rates_angle = None
all_one_rates = None
gnn_rates = None
gnn_rates_amplitude = None
for (data, d_test, c_test) in test_loader:
    K = d_test.shape[-1]
    n = len(data['UE'].x)
    bs = len(data['UE'].x) // K
    user_feats = data['AP'].x
    item_feats = data['UE'].x
    node_features = {'AP': user_feats, 'UE': item_feats}
    output = model(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs,-1)
    output1 = model_qml_amplitude(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs,-1)
    output2 = model_qml_mid_angle(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs,-1)
    gnn_rates = rate_loss(output, d_test, c_test, True).flatten().detach().numpy()
    gnn_rates_amplitude = rate_loss(output1, d_test, c_test, True).flatten().detach().numpy()
    gnn_rates_angle = rate_loss(output2, d_test, c_test, True).flatten().detach().numpy()
    full = torch.ones_like(output)
    all_one_rates = rate_loss(full, d_test, c_test, True).flatten().detach().numpy()
test_data = scipy.io.loadmat('cf_test_6_30.mat')
opt_rates = test_data['R_cf_opt_min']

In [ ]:
min_rate, max_rate = 0, 2
y_axis = np.arange(0, 1.0, 1/202)
all_one_rates.sort()
gnn_rates_angle.sort();opt_rates.sort();gnn_rates.sort();all_one_rates.sort();gnn_rates_amplitude.sort()
gnn_rates_amplitude = np.insert(gnn_rates_amplitude, 0, min_rate); gnn_rates_amplitude = np.insert(gnn_rates_amplitude,201,max_rate)
gnn_rates_angle = np.insert(gnn_rates_angle, 0, min_rate); gnn_rates_angle = np.insert(gnn_rates_angle,201,max_rate)
gnn_rates = np.insert(gnn_rates, 0, min_rate); gnn_rates = np.insert(gnn_rates,201,max_rate)
all_one_rates = np.insert(all_one_rates, 0, min_rate); all_one_rates = np.insert(all_one_rates,201,max_rate)
opt_rates = np.insert(opt_rates, 0, min_rate); opt_rates = np.insert(opt_rates,201,max_rate)

In [ ]:
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams.update({'font.size': 12})
fig, ax = plt.subplots(figsize=(6, 6), dpi=180)


ax.plot(gnn_rates, y_axis, label='GNN', linestyle = '-',)
ax.plot(gnn_rates_angle, y_axis, label='HQGNN angle', linestyle='--')
ax.plot(gnn_rates_amplitude, y_axis, label='HQGNN amplitude', linestyle=':')
ax.plot(opt_rates, y_axis, label='Optimal')
ax.plot(all_one_rates, y_axis, label='Maximum Power')


ax.set_xlabel('Minimum rate [bps/Hz]')
ax.set_ylabel('Empirical CDF')
ax.legend(loc='upper left' )
ax.grid()

# Define the zoomed area
x1, x2, y1, y2 = 1.04, 1.075, 0.5, 0.55

axins = ax.inset_axes([0.75, 0.5, 0.2, 0.2])

# Plot the same data on the inset
axins.plot(gnn_rates, y_axis, label='GNN')
axins.plot(gnn_rates_angle, y_axis, label='HQGNN angle', linestyle='--')
axins.plot(gnn_rates_amplitude, y_axis, label='HQGNN amplitude', linestyle=':')
axins.plot(opt_rates, y_axis, label='Optimal')
axins.plot(all_one_rates, y_axis, label='Maximum Power')

# Set the zoom limits on the inset
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)

# Add zoom indication lines
ax.indicate_inset_zoom(axins, edgecolor="black")

# Display the plot
plt.show()

In [ ]:
for (data, d_test, c_test) in test_loader:
    K = d_test.shape[-1]
    n = len(data['UE'].x)
    bs = len(data['UE'].x) // K
    user_feats = data['AP'].x
    item_feats = data['UE'].x
    node_features = {'AP': user_feats, 'UE': item_feats}
    output_qgnn = model_qml_mid_angle(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs,-1)
    output_gnn = model(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs, -1)
    gnn_test_rates = rate_loss(output_gnn, d_test, c_test, True).flatten().detach().numpy()
    gnn_test_rates_q = rate_loss(output_qgnn, d_test, c_test, True).flatten().detach().numpy()

In [ ]:
gnn_angle = []
gnn_mid = []
gnn = []
opt = []

for i in range(30, 91, 10):
    open_file = 'cf_test_10_' + str(i) + '_min.mat'
    data = scipy.io.loadmat(open_file)
    beta = data['betas'][:100]
    direct = data['directs'][:100]
    cross = data['corsses'][:100].transpose(0, 2, 1)
    opti = data['R_cf_opt_min'][:100]
    
    _, norm_losses = normalize_data(beta**(1/2), beta**(1/2))
    data = PCDataset(norm_losses, direct, cross, (10, i))
    loader = DataLoader(data, 100, shuffle=False, collate_fn=collate)
    
    gnn_angle_rates = []
    gnn_mid_rates = []
    gnn_rates = []
    
    for (data, d_test, c_test) in loader:
        K = d_test.shape[-1]
        n = len(data['UE'].x)
        bs = len(data['UE'].x) // K
        user_feats = data['AP'].x
        item_feats = data['UE'].x
        node_features = {'AP': user_feats, 'UE': item_feats}
        
        output_qgnn_angle = model_qml_mid_angle(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs, -1)
        output_qgnn_mid = model_qml_amplitude(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs, -1)
        output_gnn = model(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs, -1)
        
        gnn_test_rates_angle = rate_loss(output_qgnn_angle, d_test, c_test, True).flatten().detach().numpy()
        gnn_test_rates_mid = rate_loss(output_qgnn_mid, d_test, c_test, True).flatten().detach().numpy()
        gnn_test_rates = rate_loss(output_gnn, d_test, c_test, True).flatten().detach().numpy()
        
        gnn_angle_rates.append(np.mean(gnn_test_rates_angle))
        gnn_mid_rates.append(np.mean(gnn_test_rates_mid))
        gnn_rates.append(np.mean(gnn_test_rates))
    
    gnn_angle.append(np.mean(gnn_angle_rates))
    gnn_mid.append(np.mean(gnn_mid_rates))
    gnn.append(np.mean(gnn_rates))
    opt.append(np.mean(opti)) 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.family'] = 'STIXGeneral'
plt.rcParams.update({'font.size': 12})

x_labels = list(range(30, 91, 10))
x = np.arange(len(x_labels))

plt.figure(figsize=(12, 6))
width = 0.2

plt.bar(x - 1.5 * width, gnn_angle, width, label='HQGNN angle', color='#87CEEB', hatch='.', edgecolor='black', alpha=0.85)  # Xanh dương nhạt
plt.bar(x - 0.5 * width, gnn_mid, width, label='HQGNN amplitude', color='#FFB6A0', hatch='..', edgecolor='black', alpha=0.85)  # Cam nhạt
plt.bar(x + 0.5 * width, gnn, width, label='GNN', color='#98FB98', edgecolor='black', alpha=0.85)  # Xanh lá nhạt
plt.bar(x + 1.5 * width, opt, width, label='Optimal', color='#FFD700', hatch='...', edgecolor='black', alpha=0.85)  # Vàng nhạt

plt.xlabel('Number of APs', fontsize=14, fontweight='bold')
plt.ylabel('Average Rate', fontsize=14, fontweight='bold')
plt.title('Comparison of Average Rates for Different Models-10UEs', fontsize=14, fontweight='bold')
plt.xticks(x, x_labels, fontsize=14)
plt.yticks(fontsize=14)
plt.legend(fontsize=14, loc='upper left', bbox_to_anchor=(1, 1), frameon=True, edgecolor='black')

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


In [ ]:
for (data, d_test, c_test) in test_loader:
    K = d_test.shape[-1]
    n = len(data['UE'].x)
    bs = len(data['UE'].x) // K
    user_feats = data['AP'].x
    item_feats = data['UE'].x
    node_features = {'AP': user_feats, 'UE': item_feats}
    output_qgnn_angle = model_qml_mid_angle(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs,-1)
    output_qgnn_amplitude = model_qml_amplitude(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs,-1)
    output_gnn = model(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs, -1)
    gnn_test_rates = rate_loss(output_gnn, d_test, c_test, True).flatten().detach().numpy()
    qgnn_test_rates_a = rate_loss(output_qgnn_angle, d_test, c_test, True).flatten().detach().numpy()
    qgnn_test_rates_q = rate_loss(output_qgnn_amplitude, d_test, c_test, True).flatten().detach().numpy()
print(np.mean( qgnn_test_rates_a), np.mean(qgnn_test_rates_q), np.mean(gnn_test_rates))

In [ ]:
print(-test_model(test_loader, model_qml_mid_angle),-test_model(test_loader, model_qml_amplitude),-test_model(test_loader, model))

In [ ]:
import pandas as pd
models = {
    "GNN": model,
    "HQGNN Amplitude": model_qml_amplitude,
    "HQGNN Angle": model_qml_mid_angle,
}

params_count = {name: sum(p.numel() for p in model.parameters()) for name, model in models.items()}
for (data, d_test, c_test) in test_loader:
    K = d_test.shape[-1]
    n = len(data['UE'].x)
    bs = len(data['UE'].x) // K
    user_feats = data['AP'].x
    item_feats = data['UE'].x
    node_features = {'AP': user_feats, 'UE': item_feats}
    output_qgnn_angle = model_qml_mid_angle(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs,-1)
    output_qgnn_amplitude = model_qml_amplitude(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs,-1)
    output_gnn = model(node_features, data.edge_index_dict, data.edge_attr_dict).reshape(bs, -1)
    gnn_test_rates = rate_loss(output_gnn, d_test, c_test, True).flatten().detach().numpy()
    qgnn_test_rates_a = rate_loss(output_qgnn_angle, d_test, c_test, True).flatten().detach().numpy()
    qgnn_test_rates_q = rate_loss(output_qgnn_amplitude, d_test, c_test, True).flatten().detach().numpy()

avg_gnn_test_rates_angle = -test_model(test_loader, model_qml_mid_angle)
avg_gnn_test_rates_amplitude = -test_model(test_loader, model_qml_amplitude)
avg_gnn_test_rates = -test_model(test_loader, model)
test_data = scipy.io.loadmat('cf_test_6_30.mat')
opt_rates = np.mean(test_data['R_cf_opt_min'])


performance_percentages = {
    "GNN": (avg_gnn_test_rates / opt_rates) * 100,
    "HQGNN Amplitude": (avg_gnn_test_rates_amplitude / opt_rates) * 100,
    "HQGNN Angle": (avg_gnn_test_rates_angle / opt_rates) * 100,
}
data = {
    "Model": ["GNN", "HQGNN Amplitude", "HQGNN Angle"],
    "Parameter Numbers": [params_count["GNN"], params_count["HQGNN Amplitude"], params_count["HQGNN Angle"]],
    "Performance (%)": [performance_percentages["GNN"], performance_percentages["HQGNN Amplitude"], performance_percentages["HQGNN Angle"]]
}
params_df = pd.DataFrame(data)
print(params_df)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def count_parameters(model):
    quantum_params = 0
    classical_params = 0
    for name, param in model.named_parameters():
        if 'lin_qml' in name or 'lin_res' in name:  # Phân loại các tham số thuộc về Quantum
            quantum_params += param.numel()
        else:  # Các tham số khác thuộc về Classical
            classical_params += param.numel()
    return quantum_params, classical_params
models = {
    "GNN": model,
    "HQGNN Amplitude": model_qml_amplitude,
    "HQGNN Angle": model_qml_mid_angle,
}

params_count = {
    name: count_parameters(model)
    for name, model in models.items()
}





avg_gnn_test_rates_angle = -test_model(test_loader, model_qml_mid_angle)
avg_gnn_test_rates_amplitude = -test_model(test_loader, model_qml_amplitude)
avg_gnn_test_rates = -test_model(test_loader, model)

test_data = scipy.io.loadmat('cf_test_6_30.mat')
opt_rates = np.mean(test_data['R_cf_opt_min'])

# Tính hiệu suất
performance_percentages = {
    "GNN": (avg_gnn_test_rates / opt_rates) * 100,
    "HQGNN Amplitude": (avg_gnn_test_rates_amplitude / opt_rates) * 100,
    "HQGNN Angle": (avg_gnn_test_rates_angle / opt_rates) * 100,
}

# Tạo DataFrame chứa kết quả
data = {
    "Model": ["GNN", "HQGNN Amplitude", "HQGNN Angle"],
    "Classical Parameters": [params_count["GNN"][1], params_count["HQGNN Amplitude"][1], params_count["HQGNN Angle"][1]],
    "Quantum Parameters": [params_count["GNN"][0], params_count["HQGNN Amplitude"][0], params_count["HQGNN Angle"][0]],
    "Performance (%)": [performance_percentages["GNN"], performance_percentages["HQGNN Amplitude"], performance_percentages["HQGNN Angle"]]
}
params_df = pd.DataFrame(data)

fig, ax = plt.subplots(figsize=(8, 4))  # Kích thước hình ảnh
ax.axis('tight')  # Loại bỏ trục
ax.axis('off')  # Loại bỏ viền trục
table = ax.table(cellText=params_df.values, colLabels=params_df.columns, loc='center', cellLoc='center')

# Tùy chỉnh bảng
table.auto_set_font_size(False)
table.set_fontsize(12)
table.auto_set_column_width(col=list(range(len(params_df.columns))))

# Tăng khoảng cách giữa các hàng và cột
for (row, col), cell in table.get_celld().items():
    cell.set_linewidth(1.5)  # Độ dày đường kẻ
    cell.set_height(0.3)    # Tăng chiều cao hàng
    cell.set_width(0.25)    # Tăng chiều rộng cột

# Hiển thị bảng
plt.savefig("model_parameters_and_performance_table_full.png", dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
torch.save(model.state_dict(), 'model_parameters.pth')
#torch.save(model_qml_amplitude.state_dict(), 'model_amplitude_parameters.pth')
# torch.save(model_qml_mid_angle.state_dict(), 'model_angle_parameters.pth')

In [ ]:
model.load_state_dict(torch.load('model_parameters.pth'))
model_qml_amplitude.load_state_dict(torch.load('model_amplitude_parameters.pth'))
#model_qml_mid_angle.load_state_dict(torch.load('model_angle_parameters.pth'))

# Same Parameter

In [ ]:
def MLP(channels, batch_norm=True):
    return Seq(*[
        Seq(Lin(channels[i - 1], channels[i]), ReLU(), BN(channels[i]))
        for i in range(1, len(channels))
    ])
class EdgeConv_same(MessagePassing):
    def __init__(self, input_dim, node_dim, dim_out, **kwargs):
        super(EdgeConv_same, self).__init__(aggr='mean')  # mean aggregation
        self.lin = MLP([input_dim, dim_out])
        self.res_lin = Lin(node_dim, dim_out)
        self.bn = BN(dim_out)

    def forward(self, x, edge_index, edge_attr):

        feat_src, feat_dst = x


        out = self.propagate(edge_index=edge_index, x=(feat_src, feat_dst), edge_attr=edge_attr)


        return self.bn(out + self.res_lin(feat_dst))

    def message(self, x_j, x_i, edge_attr):
        # Tạo ra thông điệp từ các nút nguồn, nút đích và đặc tính cạnh
        out = torch.cat([x_j, x_i, edge_attr], dim=1)
        return self.lin(out)

    def update(self, aggr_out):
        # Cập nhật giá trị nút đích sau khi tập hợp
        return aggr_out



In [ ]:
class RGCN_same(nn.Module):
    def __init__(self):
        super(RGCN_same, self).__init__()
        self.conv1 = HeteroConv({
            ('UE', 'com-by', 'AP'): EdgeConv_same(4, 1, 13),
            ('AP', 'com', 'UE'): EdgeConv_same(4, 1, 13)
        }, aggr='mean')

        self.conv2 = HeteroConv({
            ('UE', 'com-by', 'AP'): EdgeConv_same(28, 13, 13),
            ('AP', 'com', 'UE'): EdgeConv_same(28, 13, 13)
        }, aggr='mean')

        self.conv3 = HeteroConv({
            ('UE', 'com-by', 'AP'): EdgeConv_same(28, 13, 13),
            ('AP', 'com', 'UE'): EdgeConv_same(28, 13, 13)
        }, aggr='mean')

        self.mlp = MLP([13, 20])
        self.mlp = nn.Sequential(*[self.mlp, Seq(Lin(20, 1), Sigmoid())])
    def forward(self,x_dict, edge_index_dict, edge_attr_dict):
        out = self.conv1(x_dict, edge_index_dict, edge_attr_dict)
        out = self.conv2(out, edge_index_dict, edge_attr_dict)
        out = self.conv3(out, edge_index_dict, edge_attr_dict)
        out = self.mlp(out['UE'])
        return out


In [ ]:
model_gnn = RGCN_same()
optimizer = torch.optim.Adam(model_gnn.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.65)

In [ ]:
trainable_params = sum(p.numel() for p in model_gnn.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {trainable_params}")

HQGNN amplitude

In [ ]:
import pennylane as qml
from pennylane import numpy as np

n_qubits_amplitude = 4
dev = qml.device('default.qubit', wires=n_qubits)


In [ ]:
n_layers_circuit_X = 2
def circuit_X_entangling(weights, n_qubits_amplitude):
    qml.CRX(weights[-1], wires=[n_qubits_amplitude-1, 0])
    for i in range(n_qubits_amplitude-1):
        qml.CRX(weights[i], wires=[i, (i+1)])

@qml.qnode(dev, interface='torch')
def circuit_X(inputs, layer_weights):
    qml.AmplitudeEmbedding(features=inputs, wires=range(n_qubits_amplitude), normalize=True, pad_with=0.)
    for l in range(n_layers_circuit_X):
        circuit_X_entangling(layer_weights[l], n_qubits_amplitude)
    return qml.probs(wires=range(n_qubits_amplitude))
weight_shapes_circuit_X = { "layer_weights": (n_layers_circuit_X, n_qubits_amplitude)}

In [ ]:
n_layers_circuit_Z = 2
def circuit_Z_entangling(weights, n_qubits_amplitude):
    qml.CRZ(weights[-1], wires=[n_qubits_amplitude - 1, 0])
    for i in range(n_qubits_amplitude - 1):
        qml.CRZ(weights[i], wires=[i, i + 1])

@qml.qnode(dev, interface='torch')
def circuit_Z(inputs, layer_weights):
    qml.AmplitudeEmbedding(features=inputs, wires=range(n_qubits_amplitude), normalize=True, pad_with=0.)
    for l in range(n_layers_circuit_Z):
        circuit_Z_entangling(layer_weights[l], n_qubits_amplitude)
    return qml.probs(wires=range(n_qubits_amplitude))
weight_shapes_circuit_Z = { "layer_weights": (n_layers_circuit_Z, n_qubits_amplitude)}

In [ ]:
class Q_layer_same(MessagePassing):
    def __init__(self,src_dim, dst_dim, edge_dim, **kwargs):
        super(Q_layer_same, self).__init__(aggr='mean')  # mean aggregation
        self.lin_res = qml.qnn.TorchLayer(circuit_Z, weight_shapes_circuit_Z)
        self.lin_qml = qml.qnn.TorchLayer(circuit_X, weight_shapes_circuit_X)
        self.in_linear = nn.Linear(src_dim + dst_dim + edge_dim, 2 ** n_qubits_amplitude)
        self.bn = BN(2 ** n_qubits_amplitude)

    def forward(self, x, edge_index, edge_attr):
        feat_src, feat_dst = x
        out = self.propagate(edge_index=edge_index, x=(feat_src, feat_dst), edge_attr=edge_attr)
        out = out + self.lin_res(feat_dst)
        return self.bn(out)

    def message(self, x_j, x_i, edge_attr):
        out = torch.cat([x_j, x_i, edge_attr], dim=1)
        out = self.in_linear(out)
        out = self.lin_qml(out)
        return out

    def update(self, aggr_out):
        return aggr_out

In [ ]:
class RGCN_Hybrid_amplitude_same(nn.Module):
    def __init__(self):
        super(RGCN_Hybrid_amplitude_same, self).__init__()
        out_dim = 2**n_qubits_amplitude
        edge_dim = 2

        self.conv1 = HeteroConv({
            ('UE', 'com-by', 'AP'): Q_layer_same(1, 1, edge_dim),
            ('AP', 'com', 'UE'): Q_layer_same(1, 1, edge_dim,)
        }, aggr='mean')

        self.conv2 = HeteroConv({
            ('UE', 'com-by', 'AP'): Q_layer_same(out_dim, out_dim, edge_dim),
            ('AP', 'com', 'UE'): Q_layer_same(out_dim, out_dim, edge_dim)
        }, aggr='mean')

        self.conv3 = HeteroConv({
             ('UE', 'com-by', 'AP'): Q_layer_same(out_dim, out_dim, edge_dim),
             ('AP', 'com', 'UE'): Q_layer_same(out_dim, out_dim, edge_dim)
         }, aggr='mean')

        self.mlp = MLP([16, 20])
        self.mlp = nn.Sequential(*[self.mlp, Seq(Lin(20, 1), Sigmoid())])
    def forward(self, x_dict, edge_index_dict, edge_attr_dict):
        out = self.conv1(x_dict, edge_index_dict, edge_attr_dict)
        out = self.conv2(out, edge_index_dict, edge_attr_dict)
        out = self.conv3(out, edge_index_dict, edge_attr_dict)
        out = self.mlp(out['UE'])
        return out

In [ ]:
model_amplitude_same = RGCN_Hybrid_amplitude_same().to()

optimizer_amplitude_same = torch.optim.Adam(model_amplitude_same.parameters(), lr=5e-4)
scheduler_amplitude_same = torch.optim.lr_scheduler.StepLR(optimizer_amplitude_same, step_size=10, gamma=0.7)

In [ ]:
trainable_params = sum(p.numel() for p in model_amplitude_same.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {trainable_params}")

Training

In [ ]:
record = []

for epoch in range(0, 30):
    if epoch % 1 == 0:
        with torch.no_grad():
            test_rate = test_model(test_loader, model_gnn)
            train_rate = test_model(train_loader, model_gnn)
        print(f'Epoch {epoch:03d}, Train Rate: {train_rate:.4f}, Test Rate: {test_rate:.4f}')
        record.append([train_rate, test_rate])

    train_model(epoch, model_gnn, optimizer, train_loader )
    scheduler.step()


In [ ]:
record_edge = []

for epoch in range(0, 30):

    with torch.no_grad():
        test_rate = test_model(test_loader, model_amplitude_same)
        train_rate = test_model(train_loader, model_amplitude_same)
        record_edge.append([train_rate, test_rate])
    if epoch % 1 == 0:
        print(f'Epoch {epoch:02d}, Train Rate: {train_rate:.4f}, Test Rate: {test_rate:.4f}')
    train_model(epoch, model_amplitude_same, optimizer_amplitude_same, train_loader)
    scheduler_amplitude_same.step()

In [ ]:
torch.save(model_gnn.state_dict(), 'model_gnn_same.pth')
torch.save(model_amplitude_same.state_dict(), 'model_amplitude_same.pth')
#torch.save(model_qml_mid_angle.state_dict(), 'model_angle_same.pth')

In [ ]:
# model_gnn.load_state_dict(torch.load('model_gnn_same.pth'))
model_amplitude_same.load_state_dict(torch.load('model_amplitude_same.pth'))
# model_qml_angle.load_state_dict(torch.load('model_angle_same.pth'))

In [ ]:
import pandas as pd

# Hàm để đếm tham số của mô hình
def count_parameters(model):
    quantum_params = 0
    classical_params = 0
    for name, param in model.named_parameters():
        if 'lin_qml' in name or 'lin_res' in name:  # Phân loại các tham số thuộc về Quantum
            quantum_params += param.numel()
        else:  # Các tham số khác thuộc về Classical
            classical_params += param.numel()
    return quantum_params, classical_params

models = {
    "GNN": model_gnn,
    "HQGNN Amplitude": model_amplitude_same,
    "HQGNN Angle": model_qml_mid_angle,
}

params_count = {
    name: count_parameters(model)
    for name, model in models.items()
}



avg_gnn_test_rates_angle = -test_model(test_loader, model_qml_mid_angle)
avg_gnn_test_rates_amplitude = -test_model(test_loader, model_amplitude_same)
avg_gnn_test_rates = -test_model(test_loader, model_gnn)
test_data = scipy.io.loadmat('cf_test_6_30.mat')
opt_rates = np.sum(test_data['R_cf_opt_min'])/len(test_loader.dataset)

# Hiệu suất
performance_percentages = {
    "GNN": (avg_gnn_test_rates / opt_rates) * 100,
    "HQGNN Amplitude": (avg_gnn_test_rates_amplitude / opt_rates) * 100,
    "HQGNN Angle": (avg_gnn_test_rates_angle / opt_rates) * 100,
}

# Tạo DataFrame
data = {
    "Model": ["GNN", "HQGNN Amplitude", "HQGNN Angle"],
    "Classical Parameters": [params_count["GNN"][1], params_count["HQGNN Amplitude"][1], params_count["HQGNN Angle"][1]],
    "Quantum Parameters": [params_count["GNN"][0], params_count["HQGNN Amplitude"][0], params_count["HQGNN Angle"][0]],
    
    "Performance (%)": [performance_percentages["GNN"], performance_percentages["HQGNN Amplitude"], performance_percentages["HQGNN Angle"]],
}

params_df = pd.DataFrame(data)

print(params_df)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

gnn_test_rates = None
qgnn_test_rates_a = rate_loss(output_qgnn_angle, d_test, c_test, True).flatten().detach().numpy()
qgnn_test_rates_q = rate_loss(output_qgnn_amplitude, d_test, c_test, True).flatten().detach().numpy()
def count_parameters(model):
    quantum_params = 0
    classical_params = 0
    for name, param in model.named_parameters():
        if 'lin_qml' in name or 'lin_res' in name:  
            quantum_params += param.numel()
        else:  
            classical_params += param.numel()
    return quantum_params, classical_params

models = {
    "GNN": model_gnn,
    "HQGNN Amplitude": model_amplitude_same,
    "HQGNN Angle": model_qml_mid_angle,
}

params_count = {
    name: count_parameters(model)
    for name, model in models.items()
}


avg_gnn_test_rates_angle = -test_model(test_loader, model_qml_mid_angle)
avg_gnn_test_rates_amplitude = -test_model(test_loader, model_amplitude_same)
avg_gnn_test_rates = -test_model(test_loader, model_gnn)
test_data = scipy.io.loadmat('cf_test_6_30.mat')
opt_rates = np.sum(test_data['R_cf_opt_min'])/len(test_loader.dataset)
    
performance_percentages = {
    "GNN": (avg_gnn_test_rates / opt_rates) * 100,
    "HQGNN Amplitude": (avg_gnn_test_rates_amplitude / opt_rates) * 100,
    "HQGNN Angle": (avg_gnn_test_rates_angle / opt_rates) * 100,
}


data = {
    "Model": ["GNN", "HQGNN Amplitude", "HQGNN Angle"],
    "Classical Parameters": [params_count["GNN"][1], params_count["HQGNN Amplitude"][1], params_count["HQGNN Angle"][1]],
    "Quantum Parameters": [params_count["GNN"][0], params_count["HQGNN Amplitude"][0], params_count["HQGNN Angle"][0]],
    "Performance (%)": [performance_percentages["GNN"], performance_percentages["HQGNN Amplitude"], performance_percentages["HQGNN Angle"]],
}
params_df = pd.DataFrame(data)


fig, ax = plt.subplots(figsize=(8, 4))  
ax.axis('tight')  
ax.axis('off')  
table = ax.table(cellText=params_df.values, colLabels=params_df.columns, loc='center', cellLoc='center')


table.auto_set_font_size(False)
table.set_fontsize(12)
table.auto_set_column_width(col=list(range(len(params_df.columns))))

for (row, col), cell in table.get_celld().items():
    cell.set_linewidth(1.5)  # Độ dày đường kẻ
    cell.set_height(0.3)    # Tăng chiều cao hàng
    cell.set_width(0.25) 

# Hiển thị bảng
plt.savefig("model_parameters_table_from_code.png", dpi=300, bbox_inches='tight')
plt.show()
